# Lab | Inferential statistics - T-test & P-value

## One tailed t-test:

In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file files_for_lab/machine.txt. Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
machine = pd.read_csv("/Users/Hector_Martin/Documents/Labs/week7/lab-t-tests-p-values/files_for_lab/machine.txt", encoding='utf=16', sep= '\t')

In [5]:
#Let's get all the observations per machine in 2 separate variables:

new_machine = machine['New machine']
old_machine = machine['Old machine']

#### Let's define our hypotheses based on the instructions provided above:

- H0: The new machine packs in less time
- H1: The new machine packs in more time

####  Let's define as well our Confidence and Significance levels:

- Confidence level of H0: 0.95
- Significance level of H1: 0.05

#### Now let's compute the elements required to obtain the Statistical value:

In [6]:
#We start by computing the performance mean of the new machine and old machine: 
new_machine_mean, old_machine_mean = machine['New machine'].mean(), machine['Old machine'].mean()

#Let's do the same for the Standard Deviation:
new_machine_std, old_machine_std = machine['New machine'].std(ddof=1), machine['Old machine'].std(ddof=1)

#Now the Pooled Estimate of the common Standard Deviation:
sp = ( len(new_machine) - 1 ) * ( new_machine_std**2 ) +  ( len(old_machine) - 1 ) * ( old_machine_std**2 )
sp = sp/( len(new_machine) + len(old_machine) - 2)
sp = np.sqrt(sp)

#Then the square root of the total amount of observations per each machine:
r = np.sqrt( (1/len(new_machine)) + (1/len(old_machine)) )

#And finally the Statitical value:
t = ( new_machine_mean - old_machine_mean )/ (sp * r)

print("The mean of the New machine is {:.2f}".format(new_machine_mean))
print("The mean of the Old machine is {:.2f}".format(old_machine_mean))
print("The standard deviation of the New machine is {:.2f}".format(new_machine_std))
print("The standard deviation of the Old machine is {:.2f}".format(old_machine_std))
print("The ratio of the sample variances is {:.2f} which is bigger than 0.5 and smaller than 2".format(new_machine_std/old_machine_std))
print("The t statistic is: {:.2f}".format(t))

The mean of the New machine is 42.14
The mean of the Old machine is 43.23
The standard deviation of the New machine is 0.68
The standard deviation of the Old machine is 0.75
The ratio of the sample variances is 0.91 which is bigger than 0.5 and smaller than 2
The t statistic is: -3.40


### Hypothesis Testing:

#### Let's test out hypotheses first with the Critical value: 

In [7]:
Zc = st.t.ppf(1-(0.05),df = len(new_machine)+len(old_machine)-2)
Zc

1.7340636066175354

The **critical value** is -3.40 and our **t statistic** is 1.73, therefore **we keep H0**. The New machine is faster than the old one.

#### Let's do the same with the P value: 

In [8]:
p = st.t.cdf(t,df = len(new_machine)+len(old_machine)-2)
p

0.0016055712503872579

The **p value** is 0.66 and our **significance level** is 1.73, therefore **we keep H0**. The New machine is faster than the old one.

## Matched Pairs Test:

In this challenge we will compare dependent samples of data describing our Pokemon (file files_for_lab/pokemon.csv). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment your result.

In [9]:
pokemon = pd.read_csv("/Users/Hector_Martin/Documents/Labs/week7/lab-t-tests-p-values/files_for_lab/pokemon.csv")

In [10]:
pokemon.head(5)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


#### Let's define our hypotheses based on the instructions provided above:

- H0: both Attack and Defense scores are equal
- H1: Attack and Defense scores are different



####  Let's define as well our Confidence and Significance levels:

- Confidence level of H0: 0.95
- Significance level of H1: 0.05/2 - since it is a 2 tailed test

Since we only need Attack and Defense scores we are going to remove the rest of the columns:

In [11]:
pokemon_clean = pokemon[['Attack','Defense']]

Let's get a sample of 30 observations from the Dataframe, additionally to that we will add a column that computes the difference between both columns:

In [12]:
sample = pokemon_clean.sample(30, random_state = 1)
sample['difference'] = sample['Attack']-sample['Defense']
sample.head()

,Attack,Defense,difference
8,104,78,26
510,92,75,17
175,46,34,12
735,50,58,-8
242,105,75,30


Now let's compute both Mean and Standard Deviation of the differences between Attack and Defense of our sample:

In [13]:
sample_diff_mean, sample_diff_std = sample['difference'].mean(), sample['difference'].std(ddof=1)
sample_diff_mean, sample_diff_std

(2.8, 36.32079294288603)

In [14]:
t = sample_diff_mean / ( sample_diff_std / np.sqrt(sample.shape[0]) )
print("The mean of our samples differences is: {:.2f}".format(sample_diff_mean))
print("The standard deviation of our samples differences is: {:.2f}".format(sample_diff_std))
print("Our t statistics is: {:.2f}".format(t))

The mean of our samples differences is: 2.80
The standard deviation of our samples differences is: 36.32
Our t statistics is: 0.42


### Hypothesis Testing:

#### Let's test out hypotheses first with the Critical values: 

Since we are performing a 2 tailed test let's define the Critical value on both ends:

In [15]:
#On the right side:

Zc = st.t.ppf(1-(0.05/2),df= sample.shape[0] - 1)
Zc

2.045229642132703

In [16]:
#On the left side:

neg_Zc = st.t.ppf((0.05/2),df= sample.shape[0] - 1)
neg_Zc

-2.0452296421327034

The **critical values** are -2.04 and 2.04 and our **t statistic** is 0.42, therefore **we keep H0**. There is no significant difference between Attack and Defense scores

#### Let's do the same with the P value: 

In [17]:
st.t.cdf(t,df = sample.shape[0] - 1)

0.6620194128841399

Since our **P value** of 0.66 is bigger than the **significance level** of 0.025, we keep H0.

# OPTIONAL PART | Inferential statistics - ANOVA

## Part 1:

In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on: - Null hypothesis - Alternate hypothesis - Level of significance - Test statistic - P-value - F table

## Context:

Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data anova_lab_data.xlsx file in the files_for_lab folder

- State the null hypothesis
- State the alternate hypothesis
- What is the significance level
- What are the degrees of freedom of model, error terms, and total DoF

Data was collected randomly and provided to you in the table as shown: link to the image - Data

### Let's read the data:

In [18]:
with pd.ExcelFile('files_for_lab/anova_lab_data.xlsx', engine="openpyxl") as excel:
     micro = pd.read_excel(excel)

In [19]:
#micro

### Stating the Null Hypothesis, Alternate Hypothesis and the Significance Level:

- **H0**: The Etching Rate is the same regardless of changing the power on the Plasma Beam
- **H1**: The Etching Rate is different by changing the power on the Plasma Beam
- **Significance level**: 0.05

Let's check the amount of observations per each of the power figures:

In [20]:
micro['Power '].value_counts()

160 W    5
180 W    5
200 W    5
Name: Power , dtype: int64

We can see that all power figures have 5 observations. Let's go deeper and get the mean of each power figure groups and also the amount of samples per group in a Dataframe:

In [21]:
micro_grouped = micro.groupby('Power ')['Etching Rate'].agg(power_mean ='mean',sample='size').reset_index()
micro_grouped

,Power,power_mean,sample
0,160 W,5.792,5
1,180 W,6.238,5
2,200 W,8.318,5


### What are the Degrees of Freedom, Error Terms and total DoF?

**Degrees of Freedom:**

We have 3 different groups, therefore 3 degrees of freedom


**Error Terms:**

We have got 2 Error Terms - SST and SSE. Let's calculate both:

**SST** - how much each group mean deviates from the global mean:

In [22]:
micro

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [23]:
S2t = 0
for power in micro['Power '].unique():
    ng = len(micro[micro['Power '] == power])  
    S2t  += ng * ( ( micro[micro['Power '] == power]['Etching Rate'].mean() - micro['Etching Rate'].mean() ) ** 2) 
S2t /= ( micro['Power '].nunique() - 1 ) 

print("The value of S2t is {:.2f}".format(S2t)) 

The value of S2t is 9.09


**SSE** -  how much every single value of every group deviates from the group mean

In [24]:
S2E = 0
for power in micro['Power '].unique():
    ng = len(micro[micro['Power '] == power])
    S2E += ( ng - 1 ) * np.var(micro[micro['Power '] == power]['Etching Rate'],ddof=1)
S2E /= ( len(micro) - micro['Power '].nunique() ) 

print()
print("The value of S2E is {:.2f}".format(S2E))


The value of S2E is 0.25


Now let's compute the value of **F** statistic:

In [25]:
F = S2t / S2E
print("The value of F is {:.2f}".format(F))

The value of F is 36.88


## Which is the actual number for the final DoF? 2, 12 or 14?

In [26]:
d1 = micro['Power '].nunique() - 1
d2 = len(micro) - micro['Power '].nunique()

print("Number of degrees of freedom d1: ",d1)
print("Number of degrees of freedom d2: ",d2)

Number of degrees of freedom d1:  2
Number of degrees of freedom d2:  12


### Hypothesis Testing:

#### Critical value:

In [27]:
Fc = st.f.ppf(1-0.05,dfn=d1, dfd=d2)

print("The critical value which corresponds to the alpha of 0.05 is: {:.2f}".format(Fc))

The critical value which corresponds to the alpha of 0.05 is: 3.89


As our F is bigger than the **critical value**, we reject H0.

The probability to get any F value lower or equal to out F can be obtained with the CDF:

In [28]:
st.f.cdf(F,dfn=d1, dfd=d2)

0.9999924934157276

Thus, the probability to get any value smaller or equal to F:

$$P(x \le F=36.88)= 0.9999$$

#### P-Value:

In [29]:
1 - st.f.cdf(F,dfn=d1, dfd=d2)

7.5065842723986975e-06

Therefore, the probability to get a value bigger than F is:

$$P(x > F) = 1 - P(x \le F) = 0.000007506584272358903 < 0.05$$

Therefore, we reject **H0**

In [30]:
micro['Power '].value_counts()

160 W    5
180 W    5
200 W    5
Name: Power , dtype: int64

## Part 2:

### Using the ANOVA formula:

This formula is already integrated in Scipy Stats:

In [31]:
print(st.f_oneway(micro[micro['Power '] == '160 W']['Etching Rate'],micro[micro['Power '] == '180 W']['Etching Rate'],
                  micro[micro['Power '] == '200 W']['Etching Rate']))

F_onewayResult(statistic=36.87895470100505, pvalue=7.506584272358903e-06)


We can see that the values are exactly the same as above. 